#### Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [ ]:
>>> df = spark.read.option('header', True).csv('owid-covid-data.csv')
df = df.withColumn("total_cases_per_million", df["total_cases_per_million"].cast(IntegerType()))
>>> df.select('iso_code','location','date',
'total_cases_per_million').where(df.date =="2020-03-31").sort("total_cases_per_million",ascending =False).show(15)

In [ ]:
+--------+--------------+----------+-----------------------+
|iso_code|      location|      date|total_cases_per_million|
+--------+--------------+----------+-----------------------+
|     VAT|       Vatican|2020-03-31|                   7416|
|     SMR|    San Marino|2020-03-31|                   6953|
|     AND|       Andorra|2020-03-31|                   4866|
|     LUX|    Luxembourg|2020-03-31|                   3479|
|     ISL|       Iceland|2020-03-31|                   3326|
|     ESP|         Spain|2020-03-31|                   2051|
|     CHE|   Switzerland|2020-03-31|                   1918|
|     LIE| Liechtenstein|2020-03-31|                   1783|
|     ITA|         Italy|2020-03-31|                   1749|
|     MCO|        Monaco|2020-03-31|                   1325|
|     AUT|       Austria|2020-03-31|                   1130|
|     BEL|       Belgium|2020-03-31|                   1102|
|OWID_EUN|European Union|2020-03-31|                    893|
|     DEU|       Germany|2020-03-31|                    857|
|     NOR|        Norway|2020-03-31|                    856|
+--------+--------------+----------+-----------------------+
only showing top 15 rows

#### Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию (в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [ ]:
>>> df = df.withColumn("new_cases", df["new_cases"].cast(IntegerType()))
>>> list_ = ['World','Europe','European Union','North America','United States','Asia']
>>> df.select(df.iso_code,df.location,df.date,df.new_cases).where("2020-03-22" < 
df.date).where("2020-03-30"> df.date).filter(~df.location.isin(list_)).sort("new_cases",
ascending =False).groupBy('location').sum('new_cases').sort("sum(new_cases)",ascending =False).show(10)

In [ ]:
+--------------+--------------+
|      location|sum(new_cases)|
+--------------+--------------+
|         Spain|         51342|
|         Italy|         38551|
|       Germany|         37222|
|        France|         23537|
|United Kingdom|         19559|
|          Iran|         16671|
|        Turkey|          7981|
|       Belgium|          7435|
|   Switzerland|          7355|
| South America|          7128|
+--------------+--------------+

#### Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [ ]:
>>> yesterday_df = df.select(df.iso_code,df.location,df.date,df.new_cases).where("2020-03-21"< df.date).where("2020-03-29"> 
df.date).filter(df.location =="Russia")
>>> yesterday_df = yesterday_df.select(yesterday_df.iso_code.alias("iso_code"),yesterday_df.date.
            alias("date"),yesterday_df.location.alias("location"),yesterday_df.new_cases.alias("yesterday"))
>>> now_df = df.select(df.iso_code,df.location,df.date,df.new_cases).where("2020-03-22" < df.date).where("2020-03-30">
            df.date).filter(df.location =="Russia")

In [ ]:
>>> yesterday_df = yesterday_df.withColumn('date', F.date_add(yesterday_df['date'], 1))

In [ ]:
total_df = now_df.join(yesterday_df, now_df.date == yesterday_df.date, 'outer').select(now_df.iso_code,
    now_df.location,now_df.date,now_df.new_cases,yesterday_df.yesterday)

In [ ]:
>>> total_df = total_df.withColumn('delta',  total_df.new_cases - total_df.yesterday ).orderBy('date')
>>> total_df.show(8)

In [ ]:
+--------+--------+----------+---------+---------+------+
|iso_code|location|      date|new_cases|yesterday| delta|
+--------+--------+----------+---------+---------+------+
|     RUS|  Russia|2020-03-23|     71.0|     61.0|  10.0|
|     RUS|  Russia|2020-03-24|     57.0|     71.0| -14.0|
|     RUS|  Russia|2020-03-25|    163.0|     57.0| 106.0|
|     RUS|  Russia|2020-03-26|    182.0|    163.0|  19.0|
|     RUS|  Russia|2020-03-27|    196.0|    182.0|  14.0|
|     RUS|  Russia|2020-03-28|    228.0|    196.0|  32.0|
|     RUS|  Russia|2020-03-29|    270.0|    228.0|  42.0|
+--------+--------+----------+---------+---------+------+